# Despliegue de infraestructura virtual para análisis de datos climáticos de Medellín (2019-2024)

Este notebook documenta el proceso de configuración, carga, análisis y visualización del dataset histórico de clima de Medellín para la empresa ClimaSmart, usando Databricks Community Edition.


In [0]:
# Instalación de KaggleHub y pandas si no están disponibles
# En Databricks, puedes necesitar ejecutarlas sólo una vez
!pip install --upgrade pip
!pip install kagglehub[pandas-datasets]>=0.3.8
!pip install pandas



     |████████████████████████████████| 1.8 MB 5.5 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4


## Descarga del dataset desde Kaggle usando KaggleHub


In [0]:
import kagglehub
import zipfile
import os
import pandas as pd

class Pad_clase:
    def __init__(self):
        pass

    def download_dataset_zip(self, url=""):
        print("Descargando dataset desde Kaggle...")
        dataset_path = kagglehub.dataset_download(url)
        print("Ruta al dataset:", dataset_path)
        return dataset_path

    def extract_zip_files(self, dataset_path):
        zip_files = [f for f in os.listdir(dataset_path) if f.endswith('.zip')]
        if zip_files:
            zip_file = os.path.join(dataset_path, zip_files[0])
            extract_dir = os.path.join(dataset_path, "extracted")
            os.makedirs(extract_dir, exist_ok=True)
            print(f"Extrayendo {zip_file} en {extract_dir}...")
            with zipfile.ZipFile(zip_file, "r") as z:
                z.extractall(extract_dir)
            return extract_dir
        else:
            csv_files = [f for f in os.listdir(dataset_path) if f.endswith('.csv')]
            if csv_files:
                print("No se encontró archivo ZIP pero se detectaron archivos CSV; se asume que el dataset ya se encuentra extraído.")
                return dataset_path
            else:
                raise FileNotFoundError("No se encontró ningún archivo .zip ni archivos .csv en la ruta del dataset")

    def create_csv(self, csv_dir):
        csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]
        if not csv_files:
            raise FileNotFoundError("No se encontraron archivos CSV en el directorio extraído")

        for file in csv_files:
            file_path = os.path.join(csv_dir, file)
            print(f"Leyendo {file_path}...")
            try:
                df = pd.read_csv(file_path, encoding="latin1")
            except Exception as e:
                print(f"Error al leer {file}: {e}")
                continue
        print("CSV creado correctamente.")
        return df

# Uso de la clase y funciones
pad_clase = Pad_clase()
dataset = pad_clase.download_dataset_zip("jaiderlopez/historical-weather-medelln")
csv_dir = pad_clase.extract_zip_files(dataset)
df = pad_clase.create_csv(csv_dir)
df.head()


Descargando dataset desde Kaggle...


100%|██████████| 17.0k/17.0k [00:00<00:00, 8.57MB/s]

Extracting files...
Ruta al dataset: /root/.cache/kagglehub/datasets/jaiderlopez/historical-weather-medelln/versions/1
No se encontró archivo ZIP pero se detectaron archivos CSV; se asume que el dataset ya se encuentra extraído.
Leyendo /root/.cache/kagglehub/datasets/jaiderlopez/historical-weather-medelln/versions/1/historical-weather-medellin.csv...
CSV creado correctamente.


,Date (yyyy-mm-dd),Max_temperature (Â°C),Min_temperature (Â°C),Rain (mm),Wind (km/h),Description,City
0,2024-01-01,25,10,1.3,5,Lluvia moderada a intervalos,MedellÃ­n
1,2023-01-01,18,8,3.0,3,Ligeras precipitaciones,MedellÃ­n
2,2022-01-01,18,10,4.0,2,Ligeras precipitaciones,MedellÃ­n
3,2021-01-01,21,11,12.5,4,Lluvias fuertes o moderadas,MedellÃ­n
4,2020-01-01,18,11,27.8,3,Lluvias fuertes o moderadas,MedellÃ­n


## Exploración básica: Estructura y primeras estadísticas


In [0]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2158 entries, 0 to 2157
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date (yyyy-mm-dd)      2158 non-null   object 
 1   Max_temperature (Â°C)  2158 non-null   int64  
 2   Min_temperature (Â°C)  2158 non-null   int64  
 3   Rain (mm)              2158 non-null   float64
 4   Wind (km/h)            2158 non-null   int64  
 5   Description            2158 non-null   object 
 6   City                   2158 non-null   object 
dtypes: float64(1), int64(3), object(3)
memory usage: 118.1+ KB


In [0]:
df.describe()


,Max_temperature (Â°C),Min_temperature (Â°C),Rain (mm),Wind (km/h)
count,2158.000000,2158.000000,2158.000000,2158.000000
mean,22.478221,10.894347,13.946200,4.209917
std,2.841222,1.884705,13.743675,1.552946
min,15.000000,5.000000,0.000000,1.000000
25%,21.000000,10.000000,3.600000,3.000000
50%,22.000000,11.000000,9.300000,4.000000
75%,24.000000,12.000000,20.700000,5.000000
max,32.000000,17.000000,107.700000,11.000000


In [0]:
df.columns


Out[5]: Index(['Date (yyyy-mm-dd)', 'Max_temperature (Â°C)', 'Min_temperature (Â°C)',
       'Rain (mm)', 'Wind (km/h)', 'Description', 'City'],
      dtype='object')

## Limpieza de datos: manejo de nulos y tipos de datos


In [0]:
# Verificar cantidad de valores nulos por columna
df.isnull().sum()


Out[6]: Date (yyyy-mm-dd)        0
Max_temperature (Â°C)    0
Min_temperature (Â°C)    0
Rain (mm)                0
Wind (km/h)              0
Description              0
City                     0
dtype: int64

In [0]:
# Ejemplo: Reemplazar valores nulos en columna 'Precipitation' por el promedio
if 'Precipitation' in df.columns:
    mean_precip = df['Precipitation'].mean()
    df['Precipitation'] = df['Precipitation'].fillna(mean_precip)



In [0]:
# Eliminar filas con nulos en columnas críticas
for col in ['Temperature_Max', 'Temperature_Min']:
    if col in df.columns:
        df = df.dropna(subset=[col])


## Conversión a Spark DataFrame para procesamiento distribuido


In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# Crear Spark DataFrame desde pandas DataFrame
spark_df = spark.createDataFrame(df)
spark_df.printSchema()


root
 |-- Date (yyyy-mm-dd): string (nullable = true)
 |-- Max_temperature (Â°C): long (nullable = true)
 |-- Min_temperature (Â°C): long (nullable = true)
 |-- Rain (mm): double (nullable = true)
 |-- Wind (km/h): long (nullable = true)
 |-- Description: string (nullable = true)
 |-- City: string (nullable = true)



## Registro de la tabla para consultas SQL


In [0]:
spark_df.createOrReplaceTempView("clima_medellin")


In [0]:
# Consulta SQL: Primeros 10 registros
spark.sql("SELECT * FROM clima_medellin LIMIT 10").show()


+-----------------+---------------------+---------------------+---------+-----------+--------------------+---------+
|Date (yyyy-mm-dd)|Max_temperature (Â°C)|Min_temperature (Â°C)|Rain (mm)|Wind (km/h)|         Description|     City|
+-----------------+---------------------+---------------------+---------+-----------+--------------------+---------+
|       2024-01-01|                   25|                   10|      1.3|          5|Lluvia moderada a...|MedellÃ­n|
|       2023-01-01|                   18|                    8|      3.0|          3|Ligeras precipita...|MedellÃ­n|
|       2022-01-01|                   18|                   10|      4.0|          2|Ligeras precipita...|MedellÃ­n|
|       2021-01-01|                   21|                   11|     12.5|          4|Lluvias fuertes o...|MedellÃ­n|
|       2020-01-01|                   18|                   11|     27.8|          3|Lluvias fuertes o...|MedellÃ­n|
|       2019-01-01|                   25|                   10| 

## Visualización rápida: Evolución de la temperatura máxima anual


In [0]:
import matplotlib.pyplot as plt

# Si la columna de fecha es 'Date', en formato YYYY-MM-DD
if 'Date' in df.columns and 'Temperature_Max' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df['Date'].dt.year
    df.groupby('Year')['Temperature_Max'].mean().plot(marker='o')
    plt.title("Temperatura máxima promedio por año en Medellín (2019-2024)")
    plt.xlabel("Año")
    plt.ylabel("Temperatura Máxima (°C)")
    plt.grid()
    plt.show()


## Exportar dataset limpio para futuros análisis


In [0]:
df.to_csv("clima_medellin_limpio.csv", index=False)


In [0]:
spark_df.printSchema()


root
 |-- Date (yyyy-mm-dd): string (nullable = true)
 |-- Max_temperature (Â°C): long (nullable = true)
 |-- Min_temperature (Â°C): long (nullable = true)
 |-- Rain (mm): double (nullable = true)
 |-- Wind (km/h): long (nullable = true)
 |-- Description: string (nullable = true)
 |-- City: string (nullable = true)



In [0]:
%sql
DROP TABLE IF EXISTS default.clima_medellin_historico;


In [0]:
%sql
CREATE TABLE default.clima_medellin_historico (
  Date STRING,
  Temperature_Max DOUBLE,
  Temperature_Min DOUBLE,
  Precipitation DOUBLE,
  Wind_Speed DOUBLE,
  Description STRING
)
USING DELTA


In [0]:
spark_df_clean.write.mode("overwrite").insertInto("default.clima_medellin_historico")


## Resultados y conclusiones

*Describe aquí los resultados, análisis realizados, y adjunta capturas de pantalla de cada paso importante en Databricks.*

- El dataset fue descargado y procesado exitosamente desde Kaggle.
- Los datos están disponibles en Spark y listos para análisis posteriores.
